### Instalación de dependencias
Instala las librerías `pandas` y `openpyxl` para poder leer los datos de la tabla en formato excel que se aporta con la presente notebook. Puedes hacerlo ejecutando la siguiente celda directamente (asegúrate de haber inicializado tu entorno virtual de python para esta asignatura).

In [ ]:
%pip install pandas
%pip install openpyxl

In [ ]:
import pandas as pd
import numpy as np

### Carga los datos recogidos en el fichero `datos_porfolio_optimization.xlsx`

In [ ]:
datos = pd.read_excel('datos_porfolio_optimization.xlsx',index_col=0)

In [ ]:
datos

Definimos
$x_i=1$ si y sólo si el objecto $i$ es seleccionado; $v_i$ es el valor que aporta el recurso $i$ y $c_i$, su coste. Además, $B$ es el presupuesto total del que se dispone. Entonces, el problema de la mochila se expresa como:

$$\max \sum_i x_i v_i$$
$$s.t.\; \sum x_i c_i \leq B$$

Para facilitar su resolución, primero vamos a simpliicar la restricción $\sum x_i c_i \leq B \rightarrow \sum x_i c_i = B$. Esto se corresponde con imponer que sólo se podrán elegir conjuntos de activos cuyo valor total sea exactamente igual al presupuesto disponible (lo cual no es una situación totalmente realista).

**Implementa la función QUBO asociada a este problema y resuélvela con un solver de DWave (clásico, híbrido o cuántico).**


Es aconsejable implementar por separado las contribuciones de la función de coste y las de las restricciones. Así podremos evaluar el coste real de las soluciones proporcionadas por el solver, y determinar si es una solución factible. 

In [ ]:
# Implementa la función generate_cost_matrix(datos) que genera la matriz correspondiente con la función de coste
def generate_cost_matrix(datos):
    Q_linear = np.zeros((len(datos),len(datos)))
    # Tu código aquí	
    for i,empresa in enumerate(datos.index):
        Q_linear[i,i] = -datos.loc[empresa,'ROI']

    return Q_linear

In [ ]:
# Implementa la función generate_constraint_matrix(datos,presupuesto) que genera la matriz de restricciones
def generate_constraint_matrix(datos,presupuesto):
    Q_constraint = np.zeros((len(datos),len(datos)))
    # Tu código aquí
    # Añade la restricción de presupuesto
    for i,empresa in enumerate(datos.index):
        Q_constraint[i,i] -= 2*presupuesto*datos.loc[empresa,'Coste']
        
        for j,empresa2 in enumerate(datos.index):
            Q_constraint[i,j] += datos.loc[empresa,'Coste']*datos.loc[empresa2,'Coste']

    return Q_constraint

In [ ]:
generate_constraint_matrix(datos,4)

### Crea una función que compruebe sin una solución es factible (=que cumple las restricciones)
Dada una solución (un vector de 0s y 1s), escribe una función que devuelva el valor `True` si es una solución factible (= si se cumple la restricción), y `False` en caso contrario.

In [ ]:
def check_constraint(Q_constraint, x, presupuesto):
    factible = False
    # Tu código aquí
    # Comprueba que se cumple la restricción de presupuesto

    return x@Q_constraint@x == -presupuesto*presupuesto

In [ ]:
def interpret_solution(x,datos):
    empresas = []
    for i,empresa in enumerate(datos.index):
        if x[i] == 1:
            empresas.append(empresa)
    return empresas

In [ ]:
from dimod import ExactSolver

In [ ]:
lagrange_multiplier = 10
presupuesto = 20
Q_linear = generate_cost_matrix(datos)
Q_constraint = generate_constraint_matrix(datos,presupuesto)
qubo = Q_linear + lagrange_multiplier*Q_constraint

In [ ]:
exact_solver = ExactSolver()
exact_solver.sample_qubo(qubo).first.sample

In [ ]:
sol = np.array([(val) for val in exact_solver.sample_qubo(qubo).first.sample.values()])
interpret_solution(sol,datos)

In [ ]:
check_constraint(Q_constraint, np.array([(val) for val in exact_solver.sample_qubo(qubo).first.sample.values()]), presupuesto)

In [ ]:
sol@Q_linear@sol

In [ ]:
sol@Q_constraint@sol

In [ ]:
Q_linear

In [ ]:
Q_constraint

### Crea una función que genere la matriz qubo correspondiente con la restricción de desigualdad de presupuesto
$$\sum_i coste_i \cdot x_i \leq presupuesto$$

In [ ]:
datos

In [ ]:
datos['Coste'].to_list()

In [ ]:
def generate_inequality_constraint_matrix(datos,presupuesto):
    # Calcula la cota superior del slack
    bound_slack = presupuesto-min(datos['Coste'])

    # Calcula el número de bits necesarios para representar el slack en binario
    slack_binary = np.ceil(np.log2(bound_slack)).astype(int)
    n = len(datos)+slack_binary

    # Es posible que lo más fácil sea añadir el slack como una nueva variable
    # con coste 2^j para j=0,1,...,slack_binary
    Coste = datos['Coste'].to_list() + [2**j for j in range(slack_binary)]

    Q_constraint = np.zeros((n,n))  
    for i in range(n):
        Q_constraint[i,i] -= 2*presupuesto*Coste[i]
        for j in range(n):
            Q_constraint[i,j] += Coste[i]*Coste[j]

    return Q_constraint

Q_inequality = generate_inequality_constraint_matrix(datos,presupuesto=20)

#### Para el término lineal puedes reutilizar tu implementación anterior

In [ ]:
Q_linear_inequality = np.zeros(Q_inequality.shape)

In [ ]:
Q_linear_inequality[:len(datos),:len(datos)] = generate_cost_matrix(datos)

### ¿Tienes que modificar tu función de comprobación de la restricción?

### Resuelve el problema con la restricción de desigualdad y analiza la factibilidad de las soluciones devueltas por el solver

In [ ]:
import dimod
sampler = dimod.ExactSolver()
sample = sampler.sample_qubo(lagrange_multiplier*Q_inequality+Q_linear_inequality).first.sample
sample

In [ ]:
sol = np.array([(val) for val in sample.values()])
interpret_solution(sol,datos)

In [ ]:
sol@Q_inequality@sol

In [ ]:
sol@Q_linear_inequality@sol

In [ ]:
Q_linear_inequality

In [ ]:
import dimod

### Resolución con `dimod.BinaryQuadraticModel`

In [ ]:
bqm = dimod.BinaryQuadraticModel({},{},0,dimod.BINARY)
print(bqm)

In [ ]:
bqm = dimod.BinaryQuadraticModel({},{},0,dimod.SPIN)
print(bqm)

In [ ]:
datos

In [ ]:
bqm = dimod.BinaryQuadraticModel({},{},0,dimod.BINARY)

# Añade la función de coste
for i in range(len(datos)):
    bqm.add_variable(i,-datos.loc[datos.index[i],'ROI'])

# Añade la restricción de presupuesto
terms = [(i,datos.loc[datos.index[i],'Coste']) for i in range(len(datos))]
bqm.add_linear_equality_constraint(terms, constant=-presupuesto, lagrange_multiplier=lagrange_multiplier)

In [ ]:
bqm.to_ising()[0]
# bqm.to_numpy_matrix()

In [ ]:
qubo

In [ ]:
bqm.to_numpy_matrix()

In [ ]:
qubo

In [ ]:
bqm.add_linear_inequality_constraint(terms,constant=-presupuesto, label='ineq',lb=0,ub=0,lagrange_multiplier=lagrange_multiplier)

In [ ]:
bqm.to_numpy_matrix()